### Please <B> UPVOTE </B> if you like my notebook

In [ ]:
import os
import numpy as np
import imageio
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm
import torch.optim as optim
from IPython.display import Image

import warnings
warnings.filterwarnings('ignore')

### Autoencoders : 
#### An autoencoder is a type of artificial neural network used to learn efficient data codings in an unsupervised manner. The aim of an autoencoder is to learn a representation (encoding) for a set of data, typically for dimensionality reduction, by training the network to ignore signal “noise”.

#### The simplest form of an autoencoder is a feedforward, non-recurrent neural network similar to single layer perceptrons that participate in multilayer perceptrons (MLP) – having an input layer, an output layer and one or more hidden layers connecting them – where the output layer has the same number of nodes (neurons) as the input layer, and with the purpose of reconstructing its inputs (minimizing the difference between the input and the output) instead of predicting the target value.


In [ ]:
Image(filename = '/kaggle/input/imagesforkernel/Autoencoder_fig.png')

In [ ]:
Image(filename='/kaggle/input/imagesforkernel/autoenc.png')

In [ ]:
IMG_DIR = '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/'

### Our Images 

In [ ]:
plt.figure(figsize=(15,10))
for i in range(6):
    plt.subplot(2,3,i+1)
    choose_img = np.random.choice(os.listdir(IMG_DIR))
    image_path = os.path.join(IMG_DIR,choose_img)
    image = imageio.imread(image_path)
    plt.imshow(image)

### Autoencoder Model

In [ ]:
class Autoencoders(nn.Module):
    def __init__(self):
        super().__init__()
        ### encoder
        self.conv1 = nn.Conv2d(3,64,5)
        self.maxpool = nn.MaxPool2d(2,return_indices=True)
        self.conv2 = nn.Conv2d(64,64,5)
        self.conv3 = nn.Conv2d(64,128,5)
        ### decoder
        self.deconv1 = nn.ConvTranspose2d(128,64,5)
        self.unpool = nn.MaxUnpool2d(2)
        self.deconv2 = nn.ConvTranspose2d(64,64,5)
        self.deconv3 = nn.ConvTranspose2d(64,3,5)
    
    def forward(self,x):
        x = self.conv1(x)
        x,ind1 = self.maxpool(x)
        x = self.conv2(x)
        x,ind2 = self.maxpool(x)
        x = self.conv3(x)
        
        x = self.deconv1(x)
        x = self.unpool(x,ind2)
        x = self.deconv2(x)
        x = self.unpool(x,ind1)
        x = self.deconv3(x)
        return x

### Loading and transforming Images

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_transform = torchvision.transforms.Compose([
    transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
])

train_dataloader = torch.utils.data.DataLoader(dataset=torchvision.datasets.ImageFolder('/kaggle/input/celeba-dataset/img_align_celeba/',
                                                                                       transform=train_transform),
                                              shuffle=True,batch_size=32,num_workers=0)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model = Autoencoders().to(device)

criterian = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

### Training AutoEncoders

Note: It will take approx 30 min for each epochs

In [ ]:
n_epochs = 2 ### increase number of epochs for better result
for epoch in tqdm(range(n_epochs)):
    model.train()
    iteration = 0
    for data,_ in tqdm(train_dataloader):
        optimizer.zero_grad()
        data = data.to(device)
        output = model.forward(data)
        loss = criterian(output,data)
        loss.backward()
        optimizer.step()
        if iteration%1000 == 0:
            print(f'iteration: {iteration} , loss : {loss.item()}')
    print(f'epoch: {epoch} loss: {loss.item()}')

### Save Model

In [ ]:
torch.save(model.state_dict(),'autoencoder.h5')

In [ ]:
model1 = Autoencoders()
model1.load_state_dict(torch.load('autoencoder.h5'))
model1.eval()

### Let's test our model on 1 image

In [ ]:
for data,_ in train_dataloader:
    break

In [ ]:
pred_img = model1(data)
pred_img = pred_img.detach().numpy()
pred_img = pred_img.reshape(32,224,224,3)

### Predicted Images

In [ ]:
plt.imshow(pred_img[0])
plt.show()

### Original Image

In [ ]:
new_data = data.reshape(32,224,224,3)

plt.imshow(new_data[0])
plt.show()